## Primer Práctica WebScrapping

### Herramientas

- Beautiful Soup
- Scrappy (Opcional) 
- Matlplotlib

Url = [Boletín Judicial Morelos](http://sica.tsjmorelos2.gob.mx/boletin/boletinjudicial.php)

¿Qué vamos a hacer?
- Realizaremos WebScrapping a una página del gobierno del estado de Morelos.
- Para ser más especificos al Boletin Judicial del Tribunal Superior de Justicia del Estado de Morelos

¿Qué es un boletín Judicial?

El Boletín Judicial es un instrumento de comunicación oficial que se publica diariamente en los días hábiles, en él se incluyen los acuerdos y resoluciones del día, edictos y avisos de los Juzgados de Primera Instancia, Juzgados Menores y de las Salas del Tribunal Superior de Justicia.



Primer Paso:

Identificar los componentes de la página y probar una busqueda manual.

Componentes: 
- Formulario con un método POST que pasa como parametros ->

``
{
    "opcion":"area",
    "start":"2022-12-12",
    "end":"2023-03-04",
    "dato": "Marcos",
    "distritos":"1",
    "juzgados":"1",
    "secretarias":"1"
}
``

- Respuesta en formato HTML por parte de un servidor PHP

In [21]:
url_pagina = "http://sica.tsjmorelos2.gob.mx/boletin/boletinjudicial.php"
url_metodo_post = "http://sica.tsjmorelos2.gob.mx/boletin/DT/dat_consulta.php"

In [22]:
# Parametros de la peticion
parametros = {
    "opcion": "area",
    "start": "2022-12-12",
    "end": "2023-03-04",
    "dato": "Marcos",
    "distritos": "1",
    "juzgados": "1",
    "secretarias": "1"
}

In [23]:
#Realizar una petición POST a la página url_metodo_post
import requests
from bs4 import BeautifulSoup
import re

respuesta = requests.post(url_metodo_post, data=parametros)

# Verificar que la respuesta sea correcta
if respuesta.status_code == 200:
    print("Respuesta correcta")

    # Obtenemos el contenido de la respuesta
    contenido = respuesta.content
    # Creamos un objeto BeautifulSoup
    soup = BeautifulSoup(contenido, "html.parser")
    # Como observamos anteriormente buscaremos los tr
    todos_los_tr = soup.find_all('tr')
    #longitud de los tr 
    #print(len(todos_los_tr))

    #regex del siguiente texto 8089 15-12-2022 para buscar las pubnlicaciones
    pattern = re.compile(r"([0-9]+(-[0-9]+)+)", re.IGNORECASE)

    #inicializamos los valores a guardar
    acuerdos = "" #acuerdos
    almacenamiento = [] #arreglo de diccionarios

    #Buscaremos los td en cada uno de los tr
    for tr in todos_los_tr:
        #buscamos los td
        td_disponibles = tr.find_all('td')
        #revisamos que existan dos td
        if(len(td_disponibles) == 2):
            if(pattern.match(td_disponibles[0].text)):               
                for p in td_disponibles[1].find_all('p'):
                    #buscamos los strong para obtener los acuerdos por fecha
                    if(p.find('strong')):
                        strong = (p.find('strong').text)
                        pattern_strong = re.compile(r"[A-Za-z0-9]+ (0?[1-9]|[12][0-9]|3[01])/(0?[1-9]|[1][0-2])/[0-9]+", re.IGNORECASE)
                        if(pattern_strong.match(strong)):
                            acuerdos = p.find('strong').text
                    Acuerdo =   {
                                "Acuerdo": acuerdos,
                                "Texto": p.text,
                                "Juzgado": "JUZGADO PRIMERO CIVIL - Primera Secretaria",
                                "Keyword": "Marcos"
                                }
                    #guardamos en un arreglo de diccionarios
                    
                    almacenamiento.append(Acuerdo)
                        
            else:
                print("No tiene la longitud correcta")
    import json
    with open('data.json', 'w') as outfile:
        json.dump(almacenamiento, outfile)  
    print("Archivo guardado correctamente")           
else:
    print("Error en la respuesta")

Respuesta correcta
No tiene la longitud correcta
No tiene la longitud correcta
Archivo guardado correctamente


In [24]:
#Leemos el archivo data.json y los primeros 50 registros los asignamos a un dataframe de pandas con los campos Acuerdo, Texto, Juzgado y Keyword
import pandas as pd
import json
with open('data.json') as json_file:
    data = json.load(json_file)
    df = pd.DataFrame(data , columns=['Acuerdo', 'Texto', 'Juzgado' ,  'Keyword'])
df.head(15)


,Acuerdo,Texto,Juzgado,Keyword
0,,BOLETÍN NÚM. 8089,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
1,,JUZGADO PRIMERO CIVIL DE PRIMERA INSTANCIA,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
2,,DEL PRIMER DISTRITO JUDICIAL.,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
3,,PRIMERA SECRETARIA,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
4,,,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
5,Acuerdos 14/12/2022,Acuerdos 14/12/2022,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
6,Acuerdos 14/12/2022,,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
7,Acuerdos 14/12/2022,Escrito 11580 exp.1100/89 Sindicato Union de T...,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
8,Acuerdos 14/12/2022,28/2021 Rafael Javier Aguirre Álvarez e Isabel...,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
9,Acuerdos 14/12/2022,378/2022 Folio 1906 Banco Mercantil del Norte...,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos


In [25]:
df2 = df

In [26]:
type(df2)

pandas.core.frame.DataFrame

In [27]:
indexNames = df2[ df2['Acuerdo'] == ' 0' ].index
df.drop(indexNames , inplace=True)
indexNames = df2[ df2['Texto'] == '\xa0' ].index
df.drop(indexNames , inplace=True)
indexNames = df2[ df2['Acuerdo'] == ' ' ].index
df.drop(indexNames , inplace=True)
print(df2.head(15))

                Acuerdo                                              Texto  \
0                                                        BOLETÍN NÚM. 8089   
1                               JUZGADO PRIMERO CIVIL DE PRIMERA INSTANCIA   
2                                            DEL PRIMER DISTRITO JUDICIAL.   
3                                                       PRIMERA SECRETARIA   
5   Acuerdos 14/12/2022                                Acuerdos 14/12/2022   
7   Acuerdos 14/12/2022  Escrito 11580 exp.1100/89 Sindicato Union de T...   
8   Acuerdos 14/12/2022  28/2021 Rafael Javier Aguirre Álvarez e Isabel...   
9   Acuerdos 14/12/2022  378/2022 Folio 1906 Banco  Mercantil del Norte...   
10  Acuerdos 14/12/2022  352/2022-1 Folio 1644 Alma Areli Lozano Hernán...   
11  Acuerdos 14/12/2022  370/2022 Folio 1866 Modesto Goristieta Cruz Se...   
12  Acuerdos 14/12/2022  265/2020 Folio1899. Francisco David Abraira Pi...   
14  Acuerdos 13/12/2022                                Acuerdos 

In [28]:
import pandas as pd
#iterar el dataframe df2 en Texto y buscar las palabras que coincidan con las palabras de la lista de palabras clave
#si encuentra una palabra clave en el texto, se agrega el texto a un nuevo dataframe df3
#si no encuentra ninguna palabra clave en el texto, se agrega el texto a un nuevo dataframe df4
df2 = pd.DataFrame(df2)
df2.head(15)

df3 = pd.DataFrame(columns=['Acuerdo', 'Texto', 'Juzgado' ,  'Keyword'])
df4 = pd.DataFrame(columns=['Acuerdo', 'Texto', 'Juzgado' ,  'Keyword'])

palabras_clave = ['Marcos', 'marcos', 'MARCOS']

for index, row in df2.iterrows():
    if any(word in row['Texto'] for word in palabras_clave):
        df3 = df3.append(row, ignore_index=True)
    else:
        df4 = df4.append(row, ignore_index=True)


C:\Users\Adolfo\AppData\Local\Temp\ipykernel_692\438888772.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df4 = df4.append(row, ignore_index=True)
C:\Users\Adolfo\AppData\Local\Temp\ipykernel_692\438888772.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df3.append(row, ignore_index=True)


In [29]:
df3.head()

,Acuerdo,Texto,Juzgado,Keyword
0,Acuerdos 12/12/2022,44/2021. Folio 396. Leticia Ortiz Hernandez Vs...,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
1,Acuerdos 17/01/2023,44/2021. Folio 396. Leticia Ortiz Hernandez Vs...,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
2,Acuerdos 23/01/2023,44/2021. Folio 396. Leticia Ortiz Hernandez Vs...,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
3,Acuerdos 02/02/202,44/2021. Folio 396. Leticia Ortiz Hernandez Vs...,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
4,Acuerdos 08/02/202,44/2021. Folio 396. Leticia Ortiz Hernandez Vs...,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos


In [30]:
df4.head()

,Acuerdo,Texto,Juzgado,Keyword
0,,BOLETÍN NÚM. 8089,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
1,,JUZGADO PRIMERO CIVIL DE PRIMERA INSTANCIA,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
2,,DEL PRIMER DISTRITO JUDICIAL.,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
3,,PRIMERA SECRETARIA,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos
4,Acuerdos 14/12/2022,Acuerdos 14/12/2022,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos


In [34]:
#pip install spacy

In [38]:
!python -m spacy download es_core_news_sm

     --------------------------------------- 12.9/12.9 MB 10.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
#leer las named entities de los textos de df3 en el campo Texto
#guardar las named entities en un nuevo dataframe df5
import spacy
nlp = spacy.load("es_core_news_sm")
df5 = pd.DataFrame(columns=['Acuerdo', 'Texto', 'Juzgado' ,  'Keyword', 'Named Entities'])
for index, row in df3.iterrows():
    doc = nlp(row['Texto'])
    df5 = df5.append({'Acuerdo': row['Acuerdo'], 'Texto': row['Texto'], 'Juzgado': row['Juzgado'], 'Keyword': row['Keyword'], 'Named Entities': doc.ents}, ignore_index=True)
df5.head(15)



C:\Users\Adolfo\AppData\Local\Temp\ipykernel_692\3048320141.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df5 = df5.append({'Acuerdo': row['Acuerdo'], 'Texto': row['Texto'], 'Juzgado': row['Juzgado'], 'Keyword': row['Keyword'], 'Named Entities': doc.ents}, ignore_index=True)
C:\Users\Adolfo\AppData\Local\Temp\ipykernel_692\3048320141.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df5 = df5.append({'Acuerdo': row['Acuerdo'], 'Texto': row['Texto'], 'Juzgado': row['Juzgado'], 'Keyword': row['Keyword'], 'Named Entities': doc.ents}, ignore_index=True)
C:\Users\Adolfo\AppData\Local\Temp\ipykernel_692\3048320141.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df5 = df5.append({'Acuerdo': row['Acuerdo'], 'Texto': row

,Acuerdo,Texto,Juzgado,Keyword,Named Entities
0,Acuerdos 12/12/2022,44/2021. Folio 396. Leticia Ortiz Hernandez Vs...,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos,"((44/2021), (Folio, 396), (Leticia, Ortiz, Her..."
1,Acuerdos 17/01/2023,44/2021. Folio 396. Leticia Ortiz Hernandez Vs...,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos,"((44/2021), (Folio, 396), (Leticia, Ortiz, Her..."
2,Acuerdos 23/01/2023,44/2021. Folio 396. Leticia Ortiz Hernandez Vs...,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos,"((44/2021), (Folio, 396), (Leticia, Ortiz, Her..."
3,Acuerdos 02/02/202,44/2021. Folio 396. Leticia Ortiz Hernandez Vs...,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos,"((44/2021), (Folio, 396), (Leticia, Ortiz, Her..."
4,Acuerdos 08/02/202,44/2021. Folio 396. Leticia Ortiz Hernandez Vs...,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos,"((44/2021), (Folio, 396), (Leticia, Ortiz, Her..."
5,Acuerdos 14/02/202,44/2021. Folio 396. Leticia Ortiz Hernandez Vs...,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos,"((44/2021), (Folio, 396), (Leticia, Ortiz, Her..."
6,Acuerdos 16/02/202,44/2021. Folio 396. Leticia Ortiz Hernandez Vs...,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos,"((44/2021), (Folio, 396), (Leticia, Ortiz, Her..."


In [41]:
#imprimir completo la columna Named Entities
pd.set_option('display.max_colwidth', None)
df5.head(15)

,Acuerdo,Texto,Juzgado,Keyword,Named Entities
0,Acuerdos 12/12/2022,44/2021. Folio 396. Leticia Ortiz Hernandez Vs Marcos Martinez Medina. Controversia del Orden Familiar. Auto P. (11395),JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos,"((44/2021), (Folio, 396), (Leticia, Ortiz, Hernandez, Vs, Marcos, Martinez, Medina), (Controversia, del, Orden, Familiar), (Auto, P.))"
1,Acuerdos 17/01/2023,"44/2021. Folio 396. Leticia Ortiz Hernandez Vs Marcos Martinez Medina. Controversia del Orden Familiar. Auto B. (184,185) Idem B (283)",JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos,"((44/2021), (Folio, 396), (Leticia, Ortiz, Hernandez, Vs, Marcos, Martinez, Medina), (Controversia, del, Orden, Familiar), (Auto, B.), (Idem, B))"
2,Acuerdos 23/01/2023,"44/2021. Folio 396. Leticia Ortiz Hernandez Vs Marcos Martinez Medina. Controversia del Orden Familiar. Auto B. (588,561) Idem. B. (661)",JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos,"((44/2021), (Folio, 396), (Leticia, Ortiz, Hernandez, Vs, Marcos, Martinez, Medina), (Controversia, del, Orden, Familiar), (Auto, B.))"
3,Acuerdos 02/02/202,"44/2021. Folio 396. Leticia Ortiz Hernandez Vs Marcos Martinez Medina. Controversia del Orden Familiar. Auto B. (1059) Idem. B. (1169, 1191)",JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos,"((44/2021), (Folio, 396), (Leticia, Ortiz, Hernandez, Vs, Marcos, Martinez, Medina), (Controversia, del, Orden, Familiar), (Auto, B.))"
4,Acuerdos 08/02/202,44/2021. Folio 396. Leticia Ortiz Hernandez Vs Marcos Martinez Medina. Controversia del Orden Familiar. Auto B. (1272),JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos,"((44/2021), (Folio, 396), (Leticia, Ortiz, Hernandez, Vs, Marcos, Martinez, Medina), (Controversia, del, Orden, Familiar), (Auto, B.))"
5,Acuerdos 14/02/202,44/2021. Folio 396. Leticia Ortiz Hernandez Vs Marcos Martinez Medina. Controversia del Orden Familiar. Auto B. (1517) Idem B. (1627),JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos,"((44/2021), (Folio, 396), (Leticia, Ortiz, Hernandez, Vs, Marcos, Martinez, Medina), (Controversia, del, Orden, Familiar), (Auto, B.), (Idem, B.))"
6,Acuerdos 16/02/202,44/2021. Folio 396. Leticia Ortiz Hernandez Vs Marcos Martinez Medina. Controversia del Orden Familiar. Auto B.,JUZGADO PRIMERO CIVIL - Primera Secretaria,Marcos,"((44/2021), (Folio, 396), (Leticia, Ortiz, Hernandez, Vs, Marcos, Martinez, Medina), (Controversia, del, Orden, Familiar), (Auto, B.))"
